In [17]:
%matplotlib inline

import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from glob import glob
from __future__ import unicode_literals
from IPython.core.display import display, HTML

pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_rows', 50)

In [18]:
# Global configurations
# ---------------------

# Expand the notebook to the width of the browser
display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib style
plt.style.use('seaborn-ticks')

# Use LaTeX and its body font for the diagrams' text.
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.unicode'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif']  = ['Nimbus Roman Becker No9L']

# Font sizes
SIZE = 24
plt.rc('font', size=SIZE)  # controls default text sizes
plt.rc('axes', titlesize=SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)  # fontsize of the x any y labels
plt.rc('xtick', labelsize=SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE-2)  # legend fontsize
plt.rc('figure', titlesize=SIZE)  # # size of the figure title

# Use thinner lines for axes to avoid distractions.
mpl.rcParams['axes.linewidth']    = 0.75
mpl.rcParams['xtick.major.width'] = 0.75
mpl.rcParams['xtick.minor.width'] = 0.75
mpl.rcParams['ytick.major.width'] = 0.75
mpl.rcParams['ytick.minor.width'] = 0.75
mpl.rcParams['lines.linewidth'] = 2


# Do not use a box for the legend to avoid distractions.
mpl.rcParams['legend.frameon'] = False

# Restore part of matplotlib 1.5 behavior
mpl.rcParams['patch.force_edgecolor'] = True
mpl.rcParams['errorbar.capsize'] = 3

# Helpers
# -------

# Use coordinated colors. These are the "Tableau 20" colors as 
# RGB. Each pair is strong/light. For a theory of color    
tableau20 = [(31 , 119, 180), (174, 199, 232), # blue        [ 0,1 ]
             (255, 127, 14 ), (255, 187, 120), # orange      [ 2,3 ]
             (44 , 160, 44 ), (152, 223, 138), # green       [ 4,5 ]
             (214, 39 , 40 ), (255, 152, 150), # red         [ 6,7 ]
             (148, 103, 189), (197, 176, 213), # purple      [ 8,9 ]
             (140, 86 , 75 ), (196, 156, 148), # brown       [10,11]
             (227, 119, 194), (247, 182, 210), # pink        [12,13]
             (188, 189, 34 ), (219, 219, 141), # yellow      [14,15]
             (23 , 190, 207), (158, 218, 229), # cyan        [16,17]
             (65 , 68 , 81 ), (96 , 99 , 106), # gray        [18,19]
             (127, 127, 127), (143, 135, 130), # gray        [20,21] 
             (165, 172, 175), (199, 199, 199), # gray        [22,23]
             (207, 207, 207)]                   # gray        [24]
  
# Scale the RGB values to the [0, 1] range, which is the format 
# matplotlib accepts.    
for i in range(len(tableau20)):  
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)    

# Return a single plot without right and top axes
def fig_setup():
    fig = plt.figure(figsize=(13,7))
    ax = fig.add_subplot(111)  
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False)  
    ax.get_xaxis().tick_bottom()  
    ax.get_yaxis().tick_left()
    
    return fig, ax

In [19]:
def extract_time(sid):
    
    # Matteo's Magic

    profile = pd.read_csv('./%s/%s.prof'%(sid,sid.split('/')[1].strip()), 
                          header=None,
                          names=['tstamp','sid','uid','state','event','msg'],
                          usecols=['tstamp','sid','uid','state'])

    # Keep only unit profiles
    profile = profile.dropna(subset=['uid'])
    profile = profile[profile.uid.str.contains('unit')]

    # Elminate redundant 'Done' state 
    profile.loc[profile.state == 'Done'] = profile.loc[(profile.sid.str.contains('OutputFileTransfer')) & 
                                                   (profile.state == 'Done')]
    profile = profile.dropna()
    if profile[profile.state == 'Done'].state.size != 869:
        print 'ERROR'

    # Purge useless info from session ID
    profile['sid'] = profile['sid'].apply(lambda x: x.split(':')[1])
    profile = profile.reset_index(drop=True)
    
    # Keep only execution-related states
    txs = profile[(profile.state == 'Executing') | 
                  (profile.state == 'StagingOutput') | 
                  (profile.state == 'AgentStagingOutputPending')].copy()

    # Profiles are a mess in 0.42 :( 
    # - Duplicates of state StagingOutput 
    # - spare presence of state AgentStagingOutputPending
    for uid in txs.uid.tolist():
        txs[txs.uid == uid]
        if len(txs[(txs.uid == uid) & (txs.state == 'StagingOutput')]['state'].tolist()) >= 2:
            txs = txs.drop(txs.index[(txs.uid == uid) &
                                     (txs.state == 'StagingOutput') & 
                                     (txs.sid.str.contains('Thread'))])
        if 'AgentStagingOutputPending' in txs[txs.uid == uid].state.tolist() and \
           'StagingOutput' in txs[txs.uid == uid].state.tolist():
            txs = txs.drop(txs.index[(txs.uid == uid) & 
                                      (txs.state == 'AgentStagingOutputPending')])

    # We are done with sid
    txs = txs.drop('sid', axis=1)

    # Calculate $T_x$
    txs.tstamp = pd.to_numeric(txs.tstamp, errors='coerce')
    txs = txs.pivot(index='uid', columns='state', values='tstamp')
    txs['Tx'] = txs['StagingOutput']-txs['Executing']
    
    return txs

In [20]:
def extract_namd_time(sid):
    
    stdouts = glob('%s-pilot.0000/unit.*/STDOUT'%sid)
    df = pd.DataFrame(columns=['stage','NAMD Duration'])
    df.index.name = 'uid'
    
    exec_prof = open('null-ws-1024cores-trial1/execution_profile_rp.session.two.jdakka.017403.0001.csv','r')
    read_exec_lines = exec_prof.readlines()

    for line in read_exec_lines[2:]:

        if int(line.split(',')[1].strip().split('_')[1].strip()) in [1,2,3,4,5,6]:

            uid = line.split(',')[0].strip()
            stage = line.split(',')[1].strip()
            out = '%s-pilot.0000/%s/STDOUT'%(sid, uid)
        
            f = open(out,'r')
            last_line = f.readlines()[-1:][0]
        
            namd_dur = float(last_line.split('~')[1].strip().split(',')[0].strip()[:-1])
        
            df.loc[uid] = [stage, float(namd_dur/8)]
        
    return df.sort_index()

In [21]:
rp_prof = extract_time('null-ws-1024cores-trial1/rp.session.two.jdakka.017403.0001')
namd_prof = extract_namd_time('null-ws-1024cores-trial1/rp.session.two.jdakka.017403.0001')

comb_prof = rp_prof.merge(namd_prof, right_index=True,left_index=True)
comb_prof.columns = ['Executing', 'StagingOutput','TxRP', 'Stage', 'TxNAMD']

comb_prof['StagingOutputNAMD'] = comb_prof['Executing']+comb_prof['TxNAMD']

comb_prof = comb_prof[['Stage', 'Executing', 'StagingOutput','TxRP', 'TxNAMD']]
display(comb_prof)

ERROR


IOError: [Errno 2] No such file or directory: u'null-ws-1024cores-trial1/rp.session.two.jdakka.017403.0001-pilot.0000/unit.000000/STDOUT'

In [ ]:
fig, ax = fig_setup()
comb_prof.plot(kind='line', y=['TxRP', 'TxNAMD'], ax=ax)

ax.set_xlabel('uids')
ax.set_ylabel('Time (seconds)')
ax.set_title('Tx for each unit obtained from RP profiles and NAMD logs')

# Analysis
The plot shows large discrepancies between $T_x$ as measured by RP and NAMD. These discrepancies appear to be determined by spurious but common failures to timestamp the end of the execution of the unit in RP 0.42. The following substantiate this statement:

In [ ]:
sid = 'null-ws-1024cores-trial1/rp.session.two.jdakka.017403.0001'
profile = pd.read_csv('./%s/%s.prof'%(sid,sid), 
                          header=None,
                          names=['tstamp','sid','uid','state','event','msg'],
                          usecols=['tstamp','uid','state'])

# Keep only unit profiles
profile = profile.dropna(subset=['uid'])
profile = profile[profile.uid.str.contains('unit')]

profile[profile.uid == 'unit.000049']

Note how the time stampe for the states *StagingOutput* and *Done* are the same and that the state *AgentStagingInputPending* is missing. The hypothesis is that RP missed to timestamp the end of the unit execution and timestamped *StagingOutput* and *Done* when all the units had terminated execution of after that when tearing down the agent. This hypothesis can be investigated by looking at the duration of the pilot activity ($T_r$) and comparing it to the total time spent execution units ($TTX$) calculated by using NAMD $T_x$ durations.